<a href="https://colab.research.google.com/github/rocket0l4/ML-files/blob/main/Bagging_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression

In [4]:
X,y = make_regression(n_samples = 506,n_features = 13,noise=1,random_state = 42)

In [6]:
X.shape

(506, 13)

In [7]:
X

array([[ 1.07363175,  0.13296967, -1.0265153 , ...,  0.24938368,
         0.27902153, -0.09529553],
       [-1.90780756, -0.41360553, -0.86038501, ...,  0.07203686,
         0.07748052, -0.95191846],
       [-1.51891083,  0.38837573, -2.63073014, ..., -0.05592045,
         0.00422986,  0.89130369],
       ...,
       [-0.48536355,  2.31465857,  0.08187414, ...,  0.57089051,
         0.65139125,  0.95400176],
       [ 0.74625357,  2.16325472,  0.64548418, ..., -0.81693567,
         0.02874482, -0.39333881],
       [-0.0555477 , -0.03269475,  0.38406545, ..., -2.03923218,
         1.50235705,  0.71754226]])

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

In [9]:
from sklearn.model_selection import train_test_split

In [12]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size =0.80,test_size = 0.20,random_state=123)
print('Train/Test Sets Sizes : ',X_train.shape,X_test.shape,y_train.shape,y_test.shape)

Train/Test Sets Sizes :  (404, 13) (102, 13) (404,) (102,)


In [13]:
lr = LinearRegression()
dt = DecisionTreeRegressor()
knn = KNeighborsRegressor()

In [15]:
lr.fit(X_train,y_train)

LinearRegression()

In [16]:
dt.fit(X_train,y_train)

DecisionTreeRegressor()

In [17]:
knn.fit(X_train,y_train)

KNeighborsRegressor()

In [18]:
y_pred1 = lr.predict(X_test)
y_pred2 = dt.predict(X_test)
y_pred3 = knn.predict(X_test)

In [19]:
print("R^2 score for LR",r2_score(y_test,y_pred1))
print("R^2 score for dt",r2_score(y_test,y_pred2))
print("R^2 score for knn",r2_score(y_test,y_pred3))

R^2 score for LR 0.9999498990686198
R^2 score for dt 0.49524221283918324
R^2 score for knn 0.7446582035196356


In [20]:
from sklearn.ensemble import BaggingRegressor

bag_regressor = BaggingRegressor(random_state=1)
bag_regressor.fit(X_train,y_train)

BaggingRegressor(random_state=1)

In [21]:
y_pred4 = bag_regressor.predict(X_test)

In [22]:
print("Training Cofficient of R^2 : %3f"%bag_regressor.score(X_train,y_train))
print("Testing Cofficient of R^2 : %3f"%bag_regressor.score(X_test,y_test))

Training Cofficient of R^2 : 0.950018
Testing Cofficient of R^2 : 0.789047


#Here int this code we are doing HyperParameter Tuneing by help of GridSearchCV.
#After this we are caluclating the grid.best_score and grid.best_parameteres.

- For speed up the process, we use the n_jobs = -1. Due to this the all the process will going on parllely, and take less time.

In [27]:
%%time

n_samples = X.shape[0]
n_features = X.shape[1]

params = {'estimator' : [None,LinearRegression(),KNeighborsRegressor()],
          'n_estimators': [20,50,100],
          'max_samples' : [0.5,1.0],
          'max_features' : [0.5,1.0],
          'bootstrap': [True,False],
          'bootstrap_features':[True,False]}

bagging_regressor_grid = GridSearchCV(BaggingRegressor(random_state=1,n_jobs =-1),param_grid=params,cv =3,n_jobs = -1,verbose = 1)
bagging_regressor_grid.fit(X_train,y_train)

print("Train R^2 Score : %3f"%bagging_regressor_grid.best_estimator_.score(X_train,y_train))
print("Test R^2 Score : %3f"%bagging_regressor_grid.best_estimator_.score(X_test,y_test))
print("Best R^2 Score Through Grid Search : %3f"%bagging_regressor_grid.best_score_)
print("Best Parameters :",bagging_regressor_grid.best_params_)

Fitting 3 folds for each of 144 candidates, totalling 432 fits
Train R^2 Score : 0.999950
Test R^2 Score : 0.999950
Best R^2 Score Through Grid Search : 0.999943
Best Parameters : {'bootstrap': False, 'bootstrap_features': False, 'estimator': LinearRegression(), 'max_features': 1.0, 'max_samples': 0.5, 'n_estimators': 20}
CPU times: user 1.4 s, sys: 161 ms, total: 1.56 s
Wall time: 1min 11s


#Best parameters show clearly, if we use this type of parameters we can get the best score around 99.99%.